# Deep Learning for Beginners - Programming Exercises

by Aline Sindel, Katharina Breininger and Tobias Würfl

Pattern Recognition Lab, Friedrich-Alexander-University Erlangen-Nürnberg, Erlangen, Germany 
# Exercise 4



In [ ]:
# minor set-up work
import numpy as np # we will definitely need this

# automatic reloading
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Activation Functions

Activation functions play an essential role in neural networks: They introduce non-linearity. In this programming exercise, we are going to implement two activation functions: The sigmoid and the rectified linear unit (ReLU).

### Sigmoid activation function
Historically, the Sigmoid function has played a big role in the development of neural networks. Given the motivation of biological neurons and their all-or-nothing response, the sigmoid is an obvious choice close to a true step function: It scales the input between 0 and 1, and its gradient exists everywhere.
For each element of the input, it is defined as:
\begin{equation}
\mathrm{sig}(x) = \frac{1}{1 + e^{-x}} \enspace{.}
\end{equation}

To be able to backpropagate the error through the network, we need the gradient with respect to the input: 

\begin{align}
\frac{\partial \mathrm{sig}(x)}{\partial x} &= \frac{1}{1 + e^{-x}} (1 - \frac{1}{1 + e^{-x}}) \\
                                   &= \mathrm{sig}(x) (1-\mathrm{sig}(x)) \enspace{.}
\end{align}

### ReLU activation function

While the sigmoid function is still frequently used for example in recurrent networks and as the last layer for binary segmentation/classification, it has been overtaken by the rectified linear unit (ReLU) and its variants in many other setting.
The main drawback of the sigmoid function is that its gradient is close to zero everywhere apart from a small region around the origin. This can cause the so-called vanishing gradient problem, meaning that the network will learn very slow or will stop learning completely. The ReLU is much less affected by this problem, as the output is linear for inputs $>0$:

\begin{equation}
\mathrm{relu}(x) = 
\begin{cases}
x \quad \text{if}~x > 0,\\
0 \quad \text{else}.
\end{cases}
\end{equation}

However, due to the kink at position 0, the function is not continuously differentiable. Instead, we need to compute subgradients in the backward pass:

\begin{equation}
\frac{\partial \mathrm{relu}(x)}{x} = 
\begin{cases}
1 \quad \text{if}~x > 0,\\
0 \quad \text{else}.
\end{cases}
\end{equation}

For both activation functions, we need to apply the chain rule to compute the result of the backward pass:
\begin{align}
\frac{\partial L}{\partial x} &= \frac{\partial L}{\partial f(x)} \frac{\partial f(x)}{\partial x} \enspace{,}
\end{align}
where $f(x)$ stands for any of the two functions.

### Implementation task

In the following, implement the ```Sigmoid``` and ```ReLU``` activation functions. Test your implementation by running the cell below.

In [ ]:
# %load src/layers/activation_functions_0.py
#----------------------------------
# Exercise: Activation functions
#----------------------------------
# The original python file can be reloaded by typing %load src/layers/activation_functions_0.py in the first line of this cell.
# After successfully solving this exercise, type the following command in the first line of this cell:
# %%writefile src/layers/activation_functions.py
# This will save the result to a python file, which you will need for the next exercises.
from src.base import BaseLayer, Phase

class Sigmoid(BaseLayer):
    def __init__(self):
        super().__init__()
        # TODO: Define class variable(s)
        pass
    
    def forward(self, x):
        """ Return the element-wise sigmoid of the input.
            param: x (np.ndarray): input to the activation function, of arbitrary shape
            returns (np.ndarray): element-wise sigmoid(x), of the same shape as x
        """
        # TODO: Implement forward pass of the Sigmoid, also store it as class variable
        pass
        
    def backward(self, error):
        """ Return the gradient with respect to the input.
            param: error (np.ndarray): the gradient passed down from the subsequent layer, of the same 
                   shape as x in the forward pass
            returns (np.ndarray): the gradient with respect to the previous layer, of the same shape as error 
        """
        # TODO: Implement backward pass of the Sigmoid, reuse stored variable
        pass
    

class ReLU(BaseLayer):
    def __init__(self):
        super().__init__()
        # TODO: Define class variable(s)
        pass
    
    def forward(self, x):
        """ Return the result of a ReLU activation of the input.
            param: x (np.ndarray): input to the activation function, of arbitrary shape
            returns (np.ndarray): element-wise ReLU(x), of the same shape as x
        """
        # TODO: Implement forward pass of the ReLU, store the activation to reuse it in the backward pass
        pass
    
    def backward(self, error):
        """ Return the gradient with respect to the input.
            param: error (np.ndarray): the gradient passed down from the previous layer, arbitrary shape (same as x)
            returns (np.ndarray): gradient with respect to the input, of the same shape as error 
        """
        # TODO: Implement backward pass of the ReLU, reuse stored variable
        pass

In [ ]:
%run Tests/TestActivationFunctions.py
TestReLU.ReLU = ReLU
TestSigmoid.Sigmoid = Sigmoid
unittest.main(argv=['first-arg-is-ignored'], exit=False)

## Initialization

Initialization is very critical for non-convex optimization problems, and neural networks are no exception. The most simple strategy is initialization with a constant value, which is frequently used for bias initialization. Generally, bias initialization with a constant of 0 is common, however, with ReLU as activation function, a small positive value is sensible to reduce the risk of "dying ReLUs".  

For other weights in FC layers and for weights in convolutional layers that we will look at in a bit, we need a different initialization strategy. If all weights are initialized with the same value, each node would receive the same update and training becomes impossible. One option to break this symmetry is uniform random initialization. Each element of $\mathbf{W}$ is drawn from a uniform distribution with a certain range, commonly [0, 1].

However, even with random initialization, finding the right range for weights is still tricky. If the weights are too small, activations become subsequently smaller when they are passed through the layers. Conversely, if they are too large, the signal grows which each subsequent layer. Both effects hinder effective training.

Glorot and Bengio$^1$ investigated this problem in more detail and presented a strategy to find the "sweet spot" for weight initialization that keeps the variance of the input and output gradient the same. Given certain assumptions, this can be achieved by drawing the weights from a Gaussian distribution $\mathcal{N}(0, \sigma)$ with zero mean and a standard deviation depending on the number of inputs  $n_\mathrm{in}$ and outputs  $n_\mathrm{out}$ of the layer. He et al.$^2$ showed that for ReLU activations, an adapted version is required to retain this property:

\begin{equation}
\sigma = \sqrt{\frac{2}{n_\mathrm{in}}} \enspace{.}
\end{equation}

### Implementation task

As the next task, implement the initializers ```Const```, ```UniformRandom``` and ```He``` that provide the method ```initialize``` for arbitrary weight shapes. For He initialization, the second dimension of ```weight_shape``` is assumed to be the number of input nodes. As before, run the cell below to test your implementation.

$^1$ Glorot X. and Bengio Y. Understanding the difficulty of training deep feedforward neural networks. In Proc. AISTATS, PMLR 9:249-256, 2010.

$^2$ He K. et al. Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification. In CoRR, abs/1502.01852, 2015.

In [ ]:
# %load src/layers/initializers_0.py
#----------------------------------
# Exercise: Initializers
#----------------------------------
# The original python file can be reloaded by typing %load src/layers/initializers_0.py in the first line of this cell.
# After successfully solving this exercise, type the following command in the first line of this cell:
# %%writefile src/layers/initializers.py
# This will save the result to a python file, which you will need for the next exercises.
from src.base import BaseLayer, Phase

class Initializer:
    """ Base class for initializers. """
    def initialize(self, weight_shape):
        """ Return weights initialized according to the subclass definition. 
            Required to work for arbitrary weight shapes.
            Base class. 
        """
        
        # Raises an exeption in base class.
        raise NotImplementedError('Method is not implemented')

        
class Const(Initializer):
    
    def __init__(self, value):
        """ Create a constant initializer.
            params: value (float): constant that is used for initialization of weights
        """
        # TODO: Implement
        pass

    def initialize(self, weight_shape):
        """ Return a new array of weights initialized with a constant value provided by self.value.
            param: weight_shape: shape of the new array
            returns (np.ndarray): array of the given shape
        """
        # TODO: Implement
        pass

class UniformRandom(Initializer):
    
    def initialize(self, weight_shape):
        """ Return a new array of weights initialized by drawing from a uniform distribution with range [0, 1].
            param: weight_shape: shape of new array
            returns (np.ndarray): array of the given shape
        """
        # TODO: Implement
        pass

        
class He(Initializer):
       
    def initialize(self, weight_shape):
        """ Return a new array of weights initialized according to He et al.: Delving Deep into Rectifiers.
            param: weight_shape: shape of the np.array to be returned, the second dimension is assumed to be the 
                   number of input nodes
            returns (np.ndarray): array of the given shape
        """        
        # TODO: Implement based on the equation in the description, hint: check the size of the weight array
        pass
        

In [ ]:
%run Tests/TestInitializers.py
TestInitializers.Const = Const
TestInitializers.Uniform = UniformRandom
TestInitializers.He = He
unittest.main(argv=['first-arg-is-ignored'], exit=False) 

## Convolutional layers

Convolutional layers are without doubt one of the key elements of the success of neural networks in recent years. The main idea is simple: Convolution with trainable filters. They allow to learn which features are important for a given task in a data driven manner. One of their big advantages is that they inherently consider the spatial layout of the data. The animation below shows an example of a 2-D convolution of a padded input (blue) with a $3 \times 3$ filter kernel that generates the output in green:

<figure>
<img src="files/img/same_padding_no_strides.gif" width="200">
<figcaption><center>Source: https://github.com/vdumoulin/conv_arithmetic</center></figcaption>
</figure>

In this programming exercise, we will implement a 2-D convolutional layer that is fully connected in the depth/channel direction. Accordingly, given an input with $C$ channels, each filter has a shape of $M \times N \times C$, where $M$ and $N$ describe the spacial dimensions of the filter. The number of channels of the output depends on the number of filters  $S$ in the convolutional layer.

<img src="files/img/conv_forward.png" width="400">

In the example above, the input has $C = 3$ channels and the convolutional layer has $S = 2$ filters fully connected in depth direction. Accordingly, the output has two channels.

### Forward pass in a Conv layer:
In general, the forward pass can be computed in multiple ways:

#### As a special case of a fully connected layer: Matrix multiplication
Given a fixed input size, a convolutional layer can be considered as a special case of a fully connected layer. Accordingly, we can express the forward pass using a multiplication with a sparse matrix that represents the local connections within a convolutional layer. This allows us to use the same formulas as in the forward pass for the FC layer. While this presents a rather inefficient implementation, it can help to illustrate the connection between the convolutional and the FC layer.

#### Convolution
The forward pass of a *convolutional* layer can of course also be straight forwardly implemented as a convolution. Different very efficient low-level implementations of convolutions are available, e.g., implementations that use fast Fourier transforms (FFT), generalized matrix multiplication (GEMM) or that are based on Winograd minimal filtering algorithms$^3$. In the programming exercises, we will consider a "naive" convolution where we slide a filter over the image to facilitate a better understanding, and subordinate efficiency.

#### Cross-correlation
Cross-correlation is simply a convolution without a flipped filter. For filters that are initialized randomly, we are free to use cross-correlation instead of convolution in the forward pass. In the programming exercise, we will see that it saves us a bit of kernel flipping in the backward pass.

In all cases, the bias in a convolutional layer is an element-wise addition of a scalar value for each output channel.

In this programming exercise, we will implement the third option (cross-correlation) for the forward pass!

### Backward pass in a Conv layer:

In the backward pass, we need to compute the gradient with respect to the weights of the convolutional kernel, the bias and the input, given the backpropagated error tensor $E_Y$.

#### Matrix multiplication
Like in the forward pass, we can implement the backward pass by reusing the formulas from the fully connected layer if we express the convolution as a matrix multiplication. 

#### Convolution/cross-correlation
We may want to have a detailed look at the animation above, pick up pen and paper and track which pixels of the input/weight and correspondingly which pixels of the error contribute to respective gradient. For the gradient with respect to the input, we can then see that we need flipped kernel weights in the spacial dimensions (width and height). Alternatively, if we used convolution in the forward pass, we can now apply cross-correlation, and vice versa.

Next, let's have a look at the channels: If we have $S$ kernels in the forward pass, and the input has $C$ channels, we obviously need to re-arrange the weights to $C$ kernels with $S$ channels for the backward pass. 

<img src="files/img/restacking_filters.gif" width="400">

The animation above shows that channel $c$ of $E_X$ depends only on the channel $c$ of the kernel weights. You can further see how the channels of the kernels can be recombined to compute the gradient with respect to the input. 

For the gradient with respect to the weights, you can observe that a correlation operation is necessary: First, the input has to be padded with half the kernel width. Then, each channel of the input has to be correlated with the channel $s$ of $E_Y$ to yield the gradient for kernel $s$. We have to compute

\begin{equation}
\frac{\partial L}{\partial W_{c, s}} = X_c \star E_{Y_s} \end{equation}

for $c$ in $\{1, ..., C\}$ to stack together $W_s$:

<img src="files/img/conv_back_weights.png" width="400">

If convolution was used in the forward pass, the result of this correlation represents the flipped gradient, so it has to be flipped back before an update. If correlation was used instead, we save this flipping operation. You will see that in a moment, when you implement the programming exercise. To really understand this, you may want to grab pen and paper again.

The gradient with respect to the bias can be computed by simply summing over the respective channel.

Like in the fully connected layer, the gradient for the full mini-batch is the sum of the gradient of the elements of the batch.

### Stride
<figure>
<img src="files/img/padding_strides.gif" width="200">
<figcaption><center>Source: https://github.com/vdumoulin/conv_arithmetic</center></figcaption>
</figure>

A strided convolution can be used to downsample the input. From a mathematical perspective, this can be expressed as a convolution followed by subsampling. Similarly, in the backward pass, $E_Y$ is first upsampled (introducing zeros), and then processed as before.

### Padding
In this programming exercise, we will restrict the padding strategy to "same" padding, meaning the input will be padded with zeros such that the output after the convolution has the same size as the original input.

### Reshaping
Convolutional layers inherently expect the input to have a certain spatial layout with possibly arbitrary size, which is different to FC layers that expect a vector of fixed size. There are two common ways to make these operations interoperable: 

 - Flatten the input before passing it to an FC layer
 - Have the convolutional layers reshape the input to the correct spatial layout
 
Here, we will implement the first option. To this end, a FlattenLayer is introduced with the sole purpose of reshaping the input to be compatible with FC layers. As no computation is involved, the backward pass simply consists of reversing the reshaping.

### Implementation task

In the following, implement the classes ```FlattenLayer``` and ```ConvolutionalLayer``` as described above. The necessary parameters are further described in the method documentation. 

Note: If you use 3-D convolution/correlation (which makes sense from an implementation perspective), keep in mind that you potentially need to compensate for "unnecessary" flipping in the channel dimension in your implementation. Check your implementation by running the unit tests in the subsequent cell.

$^3$ Lavin A., Gray S. Fast Algorithms for Convolutional Neural Networks. In Proc. CVPR, 2016. arXiv:1509.09308.

In [ ]:
# %load src/layers/conv_0.py
#----------------------------------
# Exercise: Convolutional layers
#----------------------------------
# The original python file can be reloaded by typing %load src/layers/conv_0.py in the first line of this cell.
# After successfully solving this exercise, type the following command in the first line of this cell:
# %%writefile src/layers/conv.py
# This will save the result to a python file, which you will need for the next exercises.
from src.base import BaseLayer, Phase
from scipy.signal import convolve, correlate #use these functions for convolution and correlation
from src.layers.fully_connected import FullyConnectedLayer #your result of the previous exercise

class FlattenLayer(BaseLayer):
    def __init__(self):
        # TODO: define the necessary class variables
        pass
    
    def forward(self, x):
        """ Return a flattened version of the input.
            param: x (np.ndarray): input, of shape [b, n_channels, p, q] where b is the batch size, 
                   n_channels is the number of channels and p x q is the image size
            returns (np.ndarray): a flattened representation of x of shape [b, v] 
                   where b is the batch size and v is the output size = n_channels * p * q
        """
        # TODO: Implement flattening of the image
        pass
    
    def backward(self, error):
        """ Return the gradient with respect to the input.
            param: error (np.ndarray): the gradient passed down from the subsequent layer, of shape [b, m],
                   where b is the batch size and m is the output size with m = n_channels * p * q from 
                   the forward pass
            returns (np.ndarray): the error with restored dimensions from the forward pass, i.e. with 
                   shape [b, n_channels, p, q] where b is the batch size, n_channels is the number of 
                   channels and p x q is the image size
        """
        # TODO: Restore the image dimensions
        pass


class ConvolutionalLayer(BaseLayer):
    
    def __init__(self, stride, kernel_shape, n_kernels, learning_rate):
        """ 
            param: stride: tuple in the form of (np, nq) which denote the subsampling factor of the 
                   convolution operation in the spatial dimensions
            param: kernel_shape: integer tuple in the form of (n_channels, m, n) where n_channels is 
                   the number of input channels and m x n is the size of the filter kernels
            param: n_kernels (int): number of kernels and therefore the number of output channels
            param: learning_rate (float): learning rate of this layer
        """
        # TODO: define the necessary class variables, hint: have a look at the input variables and consider what else 
        # you need similiar to the FC layer.
        pass 
    
    def forward(self, x):
        """ Return the result of the forward pass of the convolutional layer.
            param: x(np.ndarray): input, of shape [b, n_channels, p, q],  where b is the batch size, 
                   n_channels is the number of input channels and p x q is the image size
            returns (np.ndarray): result of the forward pass, of shape (b, n_kernels, p', q') 
                   where b is the batch size, n_kernels is the number of kernels in this layer and 
                   p' x q' is the output image size (which depends on the stride)
        """
        # TODO: Implement forward pass of the convolutional layer:        
        # (1) store the input for reuse in the backward pass
        self.X = #TODO
        
        # (2) define output tensor: 
        output_tensor = #TODO
        
        # (3) Loop over all kernels and apply cross-correlation:
        for k in #...
            kernel = #TODO: expand dims of weights
            d = #TODO: use the correlation function
            d = #subsample d, consider the stride here
            #TODO: add bias
            #TODO: store in output_tensor list
        #TODO: return output_tensor as np.ndarray (see function description)        
    
    def backward(self, error):
        """ Update the weights of this layer and return the gradient with respect to the input.
            param: error (np.ndarray): of shape (b, n_kernels, p', q') where b is the batch size, n_kernels
                   is the number of kernels and p' x q' is the spacial error size (depends on the stride)
            returns (np.ndarray): the gradient with respect to the input, of shape (b, n_channels, p, q) 
                   where b is the batch size, n_channels is the number of input channels to this layer and 
                   p x q is the image size.
        """ 
        # TODO: Implement backward pass of the convolutional layer:
        # (1) Perform upsampling of error
        err_resampled = #TODO: create empty array of size (b, n_kernels, p, q)
        #TODO: assign the values of the error variable to err_resampled by considering the stride
        #TODO: err_resampled is a placeholder, now reassign the values of err_resampled to error
        
        # (2) Gradient with respect to the bias
        self._gradient_bias = #TODO
        
        # (3) Gradient with respect to lower layers (hint: w.r.t. input)
        dx = #define variable 
        #TODO: loop over input channels and output channels (hint: kernels)
        for #...
            k = #define empty kernel list
            for #...
                #get weights at index
            k = #bring k to the right dims
            #use the convolution function here and store the result in dx
        
        # (4) Gradient with respect to the weights
        #TODO: define the pad sizes: kyhl, kyhr, kxhl, kxhr
        gradient_weights = #define, use it in the loop to store the results
        x_padded = #pad self.X using kyhl, kyhr, kxhl, kxhr with constant values
        #loop over kernels and input
        for #...
            dwh = #define
                for #...
                    #use the correlation function here and store the result in dwh
                
        #TODO: store the gradients_weights as class variable
        
        # (5) update weights and bias using the learning rate and computed results, store as class variables
        #TODO
        
        #TODO: return the gradient with respect to the input as np.ndarray (see function description)
        
    
    def get_gradient_weights(self):
        """ Returns the gradient with respect to the weights from the last call of backward() """
        # TODO: Implement the getter method, hint: class variable
        pass

    def get_gradient_bias(self):
        """ Returns the gradient with respect to the bias from the last call of backward() """
        # TODO: Implement the getter method, hint: class variable
        pass
    
    def initialize(self, weights_initializer, bias_initializer):
        """ Initializes the weights/bias of this layer with the given initializers.
            param: weights_initializer: object providing a method weights_initializer.initialize(weights_shape)
                   which will return initialized weights with the given shape
            param: bias_initializer: object providing a method bias_initializer.initialize(bias_shape) 
                   which will return an initialized bias with the given shape
        """
        # TODO: Implement. To make sure that He initialization works as intended, make sure the second dimension 
        # of weights_shape contains the number of input nodes that can be computed as n_in = n_channels * m * n
        # and reshape the weights to the correct shape afterwards.
        pass

In [ ]:
%run Tests/TestConv.py
TestConv.Conv = ConvolutionalLayer
TestConv.FullyConnected = FullyConnectedLayer
TestConv.He = He
TestConv.Constant = Const
TestConv.Flatten = FlattenLayer
unittest.main(argv=['first-arg-is-ignored'], exit=False)